In [ ]:
import openai
import os
import random
import re
import time

openai.api_key="******"

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        #temperature=0.8, 
    )
    return response.choices[0].message["content"]

In [ ]:
game_rule="""You are playing a game called "Who's Undercover," designed for five players divided into two roles: Civilians and Undercover. Each player receives a word, with Undercover players getting a slightly different but similar word compared to Civilians.

The goal is to identify the Undercover players through descriptions and voting. Each round, players describe their word with a single short sentence highlighting one characteristic, avoiding repetition of others' descriptions and without explicitly stating the word. For example, if your word is "orange," you cannot say the word "orange" directly.

After everyone has spoken, all players vote, and the person with the most votes is eliminated. Civilians aim to identify and vote out the Undercover players, while the Undercover players work together to confuse others and vote out Civilians.

The game continues until either all Undercover players are eliminated (Civilians win) or only one Civilian remains (Undercover wins)."""


game_roles=["In this round, there are five players, and you will play as Player 1. Your word is: whale. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 2. Your word is: shark. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 3. Your word is: whale. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 4. Your word is: shark. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 5. Your word is: whale. Please describe your word according to the game rules."]

In [4]:
role_order=[1,2,3,4,5]
current_role_index= 0 
history=[]
def get_next_role():
    global current_role_index
    # if current_role_index is None: 
    #     next_role=random.choice(role_order)  #random beginning
    # else:
    next_role = role_order[current_role_index]
    current_role_index = (current_role_index + 1) % len(role_order)
    return next_role


In [ ]:
#mptt
first_order="""Before describing, consider the following from a first-person perspective: How should you describe the word you received? Focus on only one feature to make its description as broad as possible. Choose one angle to describe it but avoid repeating what has already been said in the history. After reflecting, give your first-person response:"""
self_judgment="""After hearing others' descriptions, guess whether you might be the undercover. If you think you could be the undercover, honestly reflect and consider how to describe the word without revealing yourself. If you think you’re a civilian, also answer honestly. Make your self-assessment:"""
second_order="""Based on your self-assessment, consider how others might perceive your first-person response. Would they think you’re the undercover? How should you modify your statement? Think from their perspective and give your final response while hiding the word itself:"""

first_vote="""Before voting, reflect on the history: Consider the features of your word and identify whose descriptions seem similar, suggesting they might share the same word. Who’s description differs significantly?"""
vote_judgment="""Reflect on your self-assessment during the speaking phase: Do you need to adjust your judgment? Clearly determine if you are a civilian or an undercover after the round ends. Remember, there are at most two undercovers among the five players. Make your final judgment:"""
second_vote="""Based on your clear judgment, think: If you’re an undercover, whose description is most similar and could be your ally? Who should you vote for to protect yourself, even if it means voting for your ally to gain the opposing team’s trust? If you’re a civilian, whose descriptions sound more like they’re from the undercover team? Vote for one person you believe is most likely on the opposing side. Provide your final choice:"""


## speaking session

In [ ]:
#game round   ==Run it before the start of each round==
last_inquiry_time = 0
history_round=[]
speak_guesses=[]
def make_response(prompts):
    global history,last_inquiry_time
    role = get_next_role()
    # Check that the time interval is greater than 30 seconds
    current_time = time.time()
    if current_time - last_inquiry_time < 30:
        print("The time interval is less than 30 seconds, waiting...")
        time.sleep(30 - (current_time - last_inquiry_time))
    
    # Update the last time GPT was asked
    last_inquiry_time = time.time()
    
    # Combine history and current prompt into input text
    tip=f"""You are player{role}, this is the previous description record, please don't have duplicate description."""
    tip2="""Now think about each of these from the first-perspective, self-judgment, and others-perspectives:"""
    tip3="""Please summarize your first-perspective thinking, your self-judgment thinking, and your others-perspective thinking to come up with a summary thought:.... this round I've judged myself to be: ....and give your vote on the thought"""
    input_text = '\n'.join([prompts]+[tip]+history+[tip2]+[first_order]+[self_judgment]+[second_order]+[tip3])
    response=get_completion(input_text)
    speak_guess = response.splitlines()[-1]
    speak_guesses.append(f'player{role} think:{speak_guess}')
    output=game_rule+game_roles[current_role_index-1]+tip2+response+"This is a record of the previous description, please do not duplicate the description."+str(history)+"""Follow the rules carefully. Think step by step, and hide the word itself in your description! Make your final statement for this round: describe your word in one sentence."""
    
    response2=get_completion(output)
    print(input_text)
    print(f'player{role} think:')
    print(response)
    #print(output)
    print(f'player{role} said:')
    print(response2)
    history.append(f'player{role} said:{response2}')
    history_round.append(f'player{role} said:{response2}')
    #print(role)

In [ ]:
#game round   ==the second round===
last_inquiry_time = 0
history_round=[]
#speak_guesses=[]
speak_guesses2=[]
def make_response(prompts):
    global history,last_inquiry_time,vote_think
    role = get_next_role()
   
    current_time = time.time()
    if current_time - last_inquiry_time < 30:
        print("The time interval is less than 30 seconds, waiting...")
        time.sleep(30 - (current_time - last_inquiry_time))
    
    
    last_inquiry_time = time.time()
    
    player_num=f'player{role}'
    player_think = next(filter(lambda x: player_num in x, speak_guesses), None)

    tip=f"""You are player{role}, here is a record of the previous description and your self-judgment and trust relationship from the last round, think about whether or not you made a change this round?"""
    tip2="""Now think about each of these from the first-perspective, self-judgment, and others-perspectives："""
    tip3="""Please summarize your first perspective thinking, your self-judgment thinking, and your others-perspective thinking to come up with a summary thought: this round I judged myself as:"""
    input_text = '\n'.join([prompts]+[tip]+history+["Self-judgment from the previous round is as follows, and whether or not you changed your mind during this round of thinking:"]+[player_think]+[tip2]+[first_order]+[self_judgment]+[second_order]+[tip3])
    response=get_completion(input_text)
    speak_guess = response.splitlines()[-1]
    speak_guesses2.append(f'player{role} think:{speak_guess}')
    output=game_rule+game_roles[current_role_index-1]+tip2+response+"This is a record of the previous description, please do not duplicate the description."+str(history)+"""Follow the rules carefully. Think step by step, and hide the word itself in your description! Make your final statement for this round: describe your word in one sentence."""
    
    response2=get_completion(output)
    print(input_text)
    print(f'player{role} think:')
    print(response)
    #print(output)
    print(f'player{role} said:')
    print(response2)
    history.append(f'player{role} said:{response2}')
    history_round.append(f'player{role} said:{response2}')
    #print(role)

In [ ]:
#game round   ==the third round (if have)===
last_inquiry_time = 0
history_round=[]
#speak_guesses=[]
speak_guesses3=[]
def make_response(prompts):
    global history,last_inquiry_time,vote_think
    role = get_next_role()
    
    current_time = time.time()
    if current_time - last_inquiry_time < 30:
        print("The time interval is less than 30 seconds, waiting...")
        time.sleep(30 - (current_time - last_inquiry_time))
    
    
    last_inquiry_time = time.time()
    
    player_num=f'player{role}'
    player_think = next(filter(lambda x: player_num in x, speak_guesses2), None)

    tip=f"""You are player{role}, here is a record of the previous description and your self-judgment and trust relationship from the last round, think about whether or not you made a change this round?"""
    tip2="""Now think about each of these from the first-perspective, self-judgment, and others-perspectives："""
    tip3="""Please summarize your first perspective thinking, your self-judgment thinking, and your others-perspective thinking to come up with a summary thought: this round I judged myself as:"""
    input_text = '\n'.join([prompts]+[tip]+history+["Self-judgment from the previous round is as follows, and whether or not you changed your mind during this round of thinking:"]+[player_think]+[tip2]+[first_order]+[self_judgment]+[second_order]+[tip3])
    response=get_completion(input_text)
    speak_guess = response.splitlines()[-1]
    speak_guesses3.append(f'player{role} think:{speak_guess}')
    output=game_rule+game_roles[current_role_index-1]+tip2+response+"This is a record of the previous description, please do not duplicate the description."+str(history)+"""Follow the rules carefully. Think step by step, and hide the word itself in your description! Make your final statement for this round: describe your word in one sentence."""
    
    response2=get_completion(output)
    print(input_text)
    print(f'player{role} think:')
    print(response)
    #print(output)
    print(f'player{role} said:')
    print(response2)
    history.append(f'player{role} said:{response2}')
    history_round.append(f'player{role} said:{response2}')
    #print(role)

In [ ]:
# ===speaking response===
current_role_index = random.randint(0, len(role_order) - 1) #random beginning

# every round discussion
for _ in range (len(role_order)):
    game=game_rule+game_roles[current_role_index]
    #print(game)
    make_response(game)

## Voting Session

In [ ]:
# round vote
vote_think=[]
def response_vote():
    global history,votes,last_inquiry_time
    role= get_next_role()
    
    current_time = time.time()
    if current_time - last_inquiry_time < 30:
        print("The time interval is less than 30 seconds, waiting...")
        time.sleep(30 - (current_time - last_inquiry_time))
    
  
    last_inquiry_time = time.time()
    player_num=f'player{role}'
    player_speech = next(filter(lambda x: player_num in x, history_round), None)
    player_think=next(filter(lambda x: player_num in x, speak_guesses), None) #c hange the speak_guesses every round
    vote_prompts=game_rule+game_roles[current_role_index-1]+f"""You are player{role}. Below is the complete history of all statements in this game. Statements with the same number belong to the same player across different rounds. For this voting round, only the players listed below are participating, and you must not vote for any other number. The statements marked as player{role} are your own statements: {history}. Your statement for this round is: {player_speech}. Your thought process during this round is: {player_think}. Without voting for yourself, analyze the history of statements and start your response with "My word is...". 
    """+first_vote+vote_judgment+second_vote+"""
    Summarize all your thoughts during the speaking phase and provide a conclusion: This round, I determine myself as: ... I trust...
    """
    response=get_completion(vote_prompts)
    print(vote_prompts)
    print(f'Player{role} speaking in voting session:')
    print(response) 
    vote_think.append(f'player{role} said:{response}')
    output_vote=game_rule+f"""You are player{role},this is your reflection in the voting session: """+response+"""Based on the final result you got from your reflection,tell me to answer in this sentence format:I vote for player in this round.... Don't repeat other people's descriptions,don't vote for yourself,you can only vote for one person."""
    response2=get_completion(output_vote)
    print(f'Player{role} said:')
    print(response2)
    votes.append(f'player{role} said:{response2}')
    

In [ ]:
# ===voting response===
votes=[]
for _ in range(len(role_order)):
    response_vote()

In [ ]:
# ===voting results===
votes

In [ ]:
# input the undercover's number
role_order=[1,2,3,4,5]
outsider=[2,4]

In [ ]:
def check_and_exclude(response, role_order):
    # catch the number
    match = re.search(r'player\s*(\d+)\s*[\u4e00-\u9fa5]*\s*is undercover', response) 
    global game_roles
   
    captured_number = int(match.group(1))
        
    # Check to see if it is in the player list, if it is then strike it out
    if captured_number in role_order:
        role_order.remove(captured_number)
        print(f'Player{captured_number}is out！')
        remove_num=str(captured_number)
        game_roles = list(filter(lambda x: remove_num not in x, game_roles))
    
    remaining_numbers = [number for number in role_order if number not in outsider]
    remaining_outsider = [number for number in role_order if number in outsider]
    print(remaining_numbers)
    if captured_number not in outsider and len(remaining_numbers) > 1:
        return "The game continues."
    if len(remaining_numbers) == 1:
        return 'Game over! The Undercover wins!'
    if len(remaining_outsider) == 0:
        return 'Game over! The Civilian wins!'
    else:
        return "The game continues."
quit_prompt=f"""Assistant, the player with the most votes describing “I think player is undercover” from each player's mouth will be considered undercover, who do you think is undercover based on the vote type? The votes are: {votes}"""
response=get_completion(quit_prompt)

print(response)

check_and_exclude(response, role_order)

In [ ]:
# Checking the players present

game_roles